In [1]:
begin
	import Pkg
    Pkg.activate()
	#Ojo, dado que activamos Pkg, ya no estamos en un entorno de notebooks reproducibles.
end

  Activating project at `~/.julia/environments/v1.11`


In [4]:
begin
	using DataFrames
	using Dates
	using CategoricalArrays
	using DuckDB
	using CSV
	using Statistics, Distributions;	

end

In [3]:
begin
    data = CSV.read("../dataset/bank-full.csv", DataFrame)
end

Row,age,job,marital,education,default,balance,housing,loan,contact,day,month,durations,campaign,pdays,previous,poutcome,y
,Int64,String15,String15,String15,String3,Int64,String3,String3,String15,Int64,String3,Int64,Int64,Int64,Int64,String7,String3
1,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
2,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
3,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
4,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
5,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
6,35,management,married,tertiary,no,231,yes,no,unknown,5,may,139,1,-1,0,unknown,no
7,28,management,single,tertiary,no,447,yes,yes,unknown,5,may,217,1,-1,0,unknown,no
8,42,entrepreneur,divorced,tertiary,yes,2,yes,no,unknown,5,may,380,1,-1,0,unknown,no
9,58,retired,married,primary,no,121,yes,no,unknown,5,may,50,1,-1,0,unknown,no



---

# Feature Engineering

In [5]:
# Create DuckDB connection

println("🦆 Initializing DuckDB...")
db = DuckDB.DB()
conn = DuckDB.connect(db)
println("DuckDB connection established.")

# Export base data to DuckDB
DuckDB.register_table(conn, data, "banking_data")
println("✅ Exported $(nrow(data)) rows to table 'banking_data'")

🦆 Initializing DuckDB...
DuckDB connection established.
✅ Exported 45210 rows to table 'banking_data'
